In [1]:
%load_ext autoreload
# Enable autoreload for all modules
%autoreload 2

In [2]:
from iris_environments.environments import get_environment_builder
import numpy as np
import ipywidgets as widgets
from functools import partial
from pydrake.all import (RigidTransform, 
                         Rgba, 
                         Sphere, 
                         RotationMatrix, 
                         TriangleSurfaceMesh, 
                         SurfaceTriangle)

In [3]:
from iris_environments.environments import env_names
currname = '2DOFFLIPPER'
plant_builder = get_environment_builder(currname)
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7000


http://localhost:7000


In [4]:
from cspace_utils.environment_helpers import get_col_func
_offset_meshcat_2 = np.array([-1,-3.8, 0.75])
q_min = np.concatenate((plant.GetPositionLowerLimits(), np.array([-0])))
q_max =  np.concatenate((plant.GetPositionUpperLimits(),np.array([-0])))

def col_func(q):
        plant.SetPositions(plant_context, q)
        #diagram.ForcedPublish(diagram_context)
        query = plant.get_geometry_query_input_port().Eval(plant_context)
        return query.HasCollisions()

def check_collision_by_ik(q0,q1,q2, min_dist=1e-5):
    q = np.array([q0,q1])
    # if np.any(q[1:]>q_max[1:]) and np.any(np.max(q[1:])*np.ones(2)<1.1*q_max[1:]):
    #     return 0
    if np.any(q[:-1]>q_max[:-1]) :
        return 0
    # if np.any(q<q_min[1:]) and np.any(np.min(q[1:])*np.ones(2)>1.1*q_min[1:]):
    #     return 0
    if np.any(q[:-1]<q_min[:-1]) :
        return 0
    if np.any(q2!=0):
        return 1
    return 1-1.*col_func(q) 

In [5]:
import pickle, os, mcubes
scale = 1
def plot_collision_constraint(N = 50, q_min = q_min, q_max= q_max, scale = 0.33):
    if f"col_cons2d{N}.pkl" in os.listdir('../tmp'):
        with open(f"../tmp/col_cons2d{N}.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        lim_min = q_min-np.array([0.5,0.1,0])
        lim_min[2] = -0.1 
        lim_max = q_max+np.array([0.5,0.1,0])
        lim_max[2] = 0.1 
       
        print(f"min {tuple(q_min-0.6)} max {tuple(q_max+0.8)}")
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(lim_min), tuple(lim_max), N, N, 3, check_collision_by_ik, 0.5)
        with open(f"../tmp/col_cons2d{N}.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)

    tri_drake = [SurfaceTriangle(*t) for t in triangles]
    vertices *=scale
    vertices += _offset_meshcat_2.reshape(-1,3)
    meshcat.SetObject("/collision_constraint/c1",
                                    TriangleSurfaceMesh(tri_drake, vertices),
                                    Rgba(0, 0, 0, 1))
    # meshcat.SetObject("/collision_constraint/c2",
    #                                 TriangleSurfaceMesh(tri_drake, vertices),
    #                                 Rgba(0.6, 0.0, 0, 1), wireframe = True)
    
plot_collision_constraint(650, scale=scale)

In [17]:
meshcat.Set2dRenderMode(RigidTransform(RotationMatrix.MakeYRotation(-np.pi/2), np.array([0,0,1])), xmin=-20, xmax=10, ymin=-10, ymax=2)
        

In [7]:
from pydrake.all import VisualizationConfig
config = VisualizationConfig()
config.enable_alpha_sliders = True

In [8]:


q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

col_col =  Rgba(0.8, 0.0, 0, 1)    
col_free =  Rgba(0.0, 0.8, 0.5, 1) 
def showres(qvis):
    q = qvis.copy()
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/conf",
                                    Sphere(0.04),
                                    col_col)
        meshcat.SetTransform(f"/drake/visualizer/conf",
                                    RigidTransform(RotationMatrix(),
                                                    scale*np.array([q[0],q[1],0])+_offset_meshcat_2+np.array([0,0,0.3])))
    else:
        # meshcat.SetObject(f"/drake/visualizer/shunk",
        #                            Sphere(0.2),
        #                            col_free)
        meshcat.SetObject(f"/drake/visualizer/conf",
                                    Sphere(0.04),
                                    col_free)
        meshcat.SetTransform(f"/drake/visualizer/conf",
                                    RigidTransform(RotationMatrix(),
                                                    scale*np.array([q[0],q[1],0])+_offset_meshcat_2+np.array([0,0,0.3])))
    # meshcat.SetTransform(f"/drake/visualizer/shunk",
    #                                RigidTransform(RotationMatrix(),
    #                                               np.array([0,0,2])))
   
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


FloatSlider(value=0.0, description='q0', max=2.568456, min=-2.568456, step=0.001)

FloatSlider(value=0.0, description='q1', max=2.568456, min=-2.568456, step=0.001)

In [9]:
query = plant.get_geometry_query_input_port().Eval(plant_context)
pairs = query.ComputeSignedDistancePairwiseClosestPoints()

id_pairs = []
for p in pairs:
    ida = p.id_A.get_value()
    idb = p.id_B.get_value()
    coll_set = set([ida, idb])
    if coll_set not in id_pairs:
        id_pairs.append(coll_set)

In [10]:
len(id_pairs)

24

In [45]:
# def col_func(q):
#         plant.SetPositions(plant_context, q)
#         #diagram.ForcedPublish(diagram_context)
#         query = plant.get_geometry_query_input_port().Eval(plant_context)
#         return query.HasCollisions()

def check_collision_id_pair(q0,q1,q2, id_pair, min_dist=1e-5):
    q = np.array([q0, q1])
    # if np.any(q[1:]>q_max[1:]) and np.any(np.max(q[1:])*np.ones(2)<1.1*q_max[1:]):
    #     return 0
    if np.any(q[:-1]>q_max[:-1]) :
        return 1
    # if np.any(q<q_min[1:]) and np.any(np.min(q[1:])*np.ones(2)>1.1*q_min[1:]):
    #     return 0
    if np.any(q[:-1]<q_min[:-1]) :
        return 1
    if np.any(q2!=0):
        return 1
    return 1-1.*coll_check_by_id_pair(q, id_pair) 

def coll_check_by_id_pair(q, id_pair):
    plant.SetPositions(plant_context, q)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    pairs = query.ComputeSignedDistancePairwiseClosestPoints()
    for i, p in enumerate(pairs):
        ida = p.id_A.get_value()
        idb = p.id_B.get_value()
        coll_set = set([ida, idb])
        if id_pair == coll_set:
            return p.distance<=0
    return False


def plot_collision_constraint_id_pair(id_pair, color = Rgba(0,0,0,1), N = 50,  q_min = q_min, q_max= q_max,scale = 0.33, h = 0):
    id_string = str(sorted(id_pair))
    if f"col_cons2d{N}_{id_string}.pkl" in os.listdir('../tmp'):
        with open(f"../tmp/col_cons2d{N}_{id_string}.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        lim_min = q_min-np.array([0.5,0.1,0])
        lim_min[2] = -0.05 
        lim_max = q_max+np.array([0.5,0.1,0])
        lim_max[2] = 0.05 
        col_hand = partial(check_collision_id_pair, id_pair = id_pair)
        print(f"min {tuple(q_min-0.6)} max {tuple(q_max+0.8)}")
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(lim_min), tuple(lim_max), N, N, 3, col_hand, 0.5)
        with open(f"../tmp/col_cons2d{N}_{id_string}.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)
    if len(vertices):
        print(f"mesh found {len(vertices)}")
        tri_drake = [SurfaceTriangle(*t) for t in triangles]
        vertices *=scale
        vertices += _offset_meshcat_2.reshape(-1,3) + np.array([0,0,1+h*0.1])
        meshcat.SetObject(f"/collision_constraint/c1{id_string}",
                                        TriangleSurfaceMesh(tri_drake, vertices),
                                        color)
    # meshcat.SetObject("/collision_constraint/c2",
    #                                 TriangleSurfaceMesh(tri_drake, vertices),
    #                                 Rgba(0.6, 0.0, 0, 1), wireframe = True)
    
# plot_collision_constraint(650, scale=scale)

from cspace_utils.colors import generate_distinct_colors, generate_maximally_different_colors

col_id_ref = [set([63, 67]), 
              set([51, 67]), 
              set([45, 67]), 
              set([27, 57]), 
              set([21, 51])]
colors = [Rgba(c[0], c[1], c[2], 0.7) for c in generate_maximally_different_colors(len(col_id_ref))]

for i, pair in enumerate(col_id_ref):
     plot_collision_constraint_id_pair(pair, color=colors[i], N = 300, scale =1, h = i)

mesh found 39392
mesh found 44272
mesh found 45584
mesh found 4444
mesh found 11345


In [12]:
showres(np.array([-1.04,  0.08]))

False

In [46]:
from pydrake.all import IrisInConfigurationSpace, IrisOptions
qseed = np.array([-1.04,  0.08])
showres(qseed)
plant.SetPositions(plant_context,qseed )
io = IrisOptions()
io.configuration_space_margin = 0.1
#io.configuration_space_margin = 0.000015
io.num_collision_infeasible_samples = 200
io.iteration_limit =4
region = IrisInConfigurationSpace(plant, plant_context, io)
print(region.A().shape)

INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 1
INFO:drake:IrisInConfigurationSpace iteration 2
INFO:drake:IrisInConfigurationSpace: Terminating because the hyperellipsoid volume change -0.06004968142108069 is below the threshold 0.02.


(12, 2)


In [51]:
#extend edges
meshcat.Delete('hplanes')
meshcat.Delete('reg')
meshcat.SetProperty('/Lights/PointLightNegativeX', 'visible', False)
meshcat.SetProperty('/Lights/PointLightPositiveX', 'visible', False)
meshcat.SetProperty('/Lights/AmbientLight/<object>', 'intensity', 5.0)
from cspace_utils.plotting import plot_vpoly_2d_meshcat, plot_point, stretch_array_to_3d
from pydrake.all import Rgba, VPolytope, HPolyhedron
dom = VPolytope(HPolyhedron.MakeBox(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()))
Hdom = HPolyhedron.MakeBox(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits())
#plot_vpoly_2d_meshcat(meshcat, vpoly=dom, name = 'dom', translation=_offset_meshcat_2+np.array([0,0,0.3]), color=Rgba(0,0,0,1),size = 0.02, axes=0)
plot_vpoly_2d_meshcat(meshcat, 
                      vpoly=VPolytope(region), 
                      name = 'reg', 
                      translation=_offset_meshcat_2+np.array([0,0,0.5]), 
                      size = 0.02, 
                      color=Rgba(0.7,0.0,0,1), 
                      axes=0)
from cspace_utils.plotting import sorted_vertices, plot_edges
v = sorted_vertices(VPolytope(region)).T#s
v = np.concatenate((v, v[0,:].reshape(1,-1)), axis=0)
edges = [ [v[i, :], v[i+1, :]]for i in range(len(v)-1)]

edges_extended = []
sidelen = np.max(plant.GetPositionUpperLimits()-plant.GetPositionLowerLimits())*np.sqrt(2)

for e in edges:
    pt1 = e[0]
    pt2 = e[1]
    dir = (pt2-pt1)/(1e-6 + np.linalg.norm(pt2-pt1))
    N = 100
    pt1_ext = pt1
    pt2_ext = pt2
    tvals = np.linspace(0, 1, 100)[::-1]
    for t in tvals:
        if Hdom.PointInSet(pt1 + dir*sidelen*t):
            pt2_ext = pt1 + dir*sidelen*t
            break
    for t in tvals:
        if Hdom.PointInSet(pt1 - dir*sidelen*t):
            pt1_ext = pt1 - dir*sidelen*t
            break
    edges_extended.append([pt1_ext, pt2_ext])

plot_edges(meshcat, edges_extended, 'hplanes/', Rgba(0,0.5,1,1),size = 0.01, translation=_offset_meshcat_2+np.array([0,0,0.3]))
plot_point(stretch_array_to_3d(qseed)+_offset_meshcat_2, meshcat, 'seed', Rgba(0.1, 0 ,0,1), 0.05)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [21]:
showres(np.zeros(2))

True

In [ ]:
#make configuration file 
import os
seed_point_file = 'benchmarks/seedpoints/'+currname+'.yml'
if seed_point_file.split('/')[2] in os.listdir('benchmarks/seedpoints'):
    file = open(seed_point_file, 'a') 
else:
    file = open(seed_point_file, 'w')
    file.write('seedpoints:\n')

# Run this cell and then use the sliders and the button to save the seed points

In [ ]:


q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 
def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/shunk",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,2])))
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

def write_seed_point_to_file(button):
    col = showres(q)
    if not col:
        line  = '- ['
        for a in q[:-1]:
            line+= str(a)+', '
        line+= str(q[-1])+']\n'
        file.write(line)
        file.flush()
        #write seedpoint to file
    else:
        raise ValueError("That point is in collision")

button = widgets.Button(description="Save")

# Attach the function to the button's click event
button.on_click(write_seed_point_to_file)

# Display the button
display(button)

In [ ]:
q = np.array([-8.30000e-03,  1.16796e+00,  9.16740e-01,  8.10270e-01,
        8.84370e-01,  1.15137e+00,  6.28890e-01,  6.30620e-01,
       -3.00000e-04,  3.74960e-01,  1.42774e+00,  5.89270e-01,
       -1.43300e-01,  5.32960e-01,  9.91740e-01,  6.33270e-01])

In [ ]:
file.close()

# Inspect seed points

In [ ]:
import yaml
import time

with open(seed_point_file, 'r') as f:
    seed_points = yaml.safe_load(f)
seed_points = np.array(seed_points['seedpoints'])

for i, s in enumerate(seed_points):
    showres(s)
    print(f" point {i+1} / {len(seed_points)}")
    time.sleep(2)


In [ ]:
plant.GetBodyByName("")

In [ ]:
showres(seed_points[-3])

In [ ]:
q